# Análise Exploratória


## Descrição
Neste notebook consta a análise exploratória da base de dados utilizada no projeto *Correlação de dados de imagens de RM e dados genéticos em paciente com Esclerose Lateral Amiotrófica* para a disciplina *Ciência e Visualização de Dados em Saúde* da Universidade Estadual de Campinas, Unicamp.

## Bibliotecas

In [92]:
# Import libraries
## Basic
import numpy as np
import scipy as sp
import pandas as pd
import random

## Graph
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from seaborn_qqplot import pplot

## Machine Learning
import statsmodels.api as sm
#from statsmodels.formula.api import ols
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

## Metrics
from sklearn.metrics import r2_score
#from scipy.stats import shapiro
from sklearn import metrics

## Inportação de Dados

In [93]:
# Import data
## Path to file
path = "../data/raw/DTI_MultAtlas.xlsx"

## Sheets names
faPath = 'FA'
l1Path = 'l1'
l2Path = 'l2'
l3Path = 'l3'
vlPath = 'volumeLabels'

## Read each excel sheet
faDataRaw = pd.read_excel(path, sheet_name = faPath)
l1DataRaw = pd.read_excel(path, sheet_name = l1Path)
l2DataRaw = pd.read_excel(path, sheet_name = l2Path)
l3DataRaw = pd.read_excel(path, sheet_name = l3Path)
vlDataRaw = pd.read_excel(path, sheet_name = vlPath)
vlDataRaw.head()

,ID/Labls,SUPERIOR PARIETAL GYRUS left (gm),CINGULATE GYRUS left (gm),SUPERIOR FRONTAL GYRUS left (gm),MIDDLE FRONTAL GYRUS left (gm),INFERIOR FRONTAL GYRUS left (gm),PRECENTRAL GYRUS left (gm),POSTCENTRAL GYRUS left (gm),ANGULAR GYRUS left (gm),PRE-CUNEUS left (gm),...,SLF-tLeft,SLFF-tRight,ICP-cerebellumLeft,ICP-cerebellumRight,CerebellumBranch-ALeft,CerebellumBranch-ARight,CerebellumBranch-BLeft,CerebellumBranch-BRight,CSF,Unused
0,c9o_02,12362,7084,16584,11484,13216,17806,8200,3140,1820,...,3888,2056,470,472,2828,2916,264,370,319946,94.0
1,c9o_03,7348,4466,10238,6452,7454,9370,4710,2136,1066,...,2822,1392,260,296,2082,1766,422,420,230842,NaN
2,c9o_04,9602,7164,14782,11700,7882,13954,5848,2262,810,...,4228,1754,216,326,2382,2712,398,474,262602,40.0
3,c9o_05,9284,7274,13514,8108,9264,11830,6120,4064,1672,...,3732,1818,256,316,2292,2118,374,320,226770,4.0
4,c9o_06,8294,6598,10790,9498,7964,10112,5232,2158,1196,...,2306,1776,262,224,2006,1812,240,286,212792,2.0


## Sumário dos Dados

In [94]:
# Data Summary 2
## Data types
print("Dataset: FA")
print(faDataRaw.dtypes)

print("\n")

print("Dataset: L1")
print(l1DataRaw.dtypes)

print("\n")

print("Dataset: L2")
print(l2DataRaw.dtypes)

print("\n")

print("Dataset: L3")
print(l3DataRaw.dtypes)

print("\n")

print("Dataset: Volume Labels")
print(vlDataRaw.dtypes)

Dataset: FA
ID/Labls                               object
SUPERIOR PARIETAL GYRUS left  (gm)    float64
CINGULATE GYRUS left  (gm)            float64
SUPERIOR FRONTAL GYRUS left (gm)      float64
MIDDLE FRONTAL GYRUS left (gm)        float64
                                       ...   
CerebellumBranch-ARight               float64
CerebellumBranch-BLeft                float64
CerebellumBranch-BRight               float64
CSF                                   float64
Unused                                float64
Length: 170, dtype: object


Dataset: L1
ID/Labls                               object
SUPERIOR PARIETAL GYRUS left  (gm)    float64
CINGULATE GYRUS left  (gm)            float64
SUPERIOR FRONTAL GYRUS left (gm)      float64
MIDDLE FRONTAL GYRUS left (gm)        float64
                                       ...   
CerebellumBranch-ARight               float64
CerebellumBranch-BLeft                float64
CerebellumBranch-BRight               float64
CSF                        

In [95]:
# Data Summary 2
## Summarry Statistics
print("Summary: FA")
print(faDataRaw.describe().transpose())

print("\n")

print("Summary: L1")
print(l1DataRaw.describe().transpose())

print("\n")

print("Summary: L2")
print(l2DataRaw.describe().transpose())

print("\n")

print("Summary: L3")
print(l3DataRaw.describe().transpose())

print("\n")

print("Summary: L3")
print(vlDataRaw.describe().transpose())

Summary: FA
                                    count      mean       std       min  \
SUPERIOR PARIETAL GYRUS left  (gm)   87.0  0.444349  0.020059  0.391171   
CINGULATE GYRUS left  (gm)           87.0  0.341726  0.008848  0.316205   
SUPERIOR FRONTAL GYRUS left (gm)     87.0  0.387573  0.016413  0.341545   
MIDDLE FRONTAL GYRUS left (gm)       87.0  0.379601  0.014524  0.342091   
INFERIOR FRONTAL GYRUS left (gm)     87.0  0.420632  0.020671  0.344214   
...                                   ...       ...       ...       ...   
CerebellumBranch-ARight              87.0  0.370548  0.029269  0.313544   
CerebellumBranch-BLeft               87.0  0.381883  0.033387  0.306729   
CerebellumBranch-BRight              87.0  0.384852  0.035752  0.310387   
CSF                                  87.0  0.150976  0.012091  0.122123   
Unused                               82.0  0.224499  0.059865  0.093848   

                                         25%       50%       75%       max  
SUPERIOR P

## Dimensões e Dados Faltantes

In [96]:
def dataShape(dataset):
    rows = dataset.shape[0]
    columns = dataset.shape[1]
    missing = dataset.isnull().sum().sum()
    
    return rows, columns, missing

In [97]:
faRows, faColumns, faMissing = dataShape(faDataRaw)
l1Rows, l1Columns, l1Missing = dataShape(l1DataRaw)
l2Rows, l2Columns, l2Missing = dataShape(l2DataRaw)
l3Rows, l3Columns, l3Missing = dataShape(l3DataRaw)
vlRows, vlColumns, vlMissing = dataShape(vlDataRaw)

In [98]:
# Summary
dataSummary = pd.DataFrame({'Data' : ['FA', 'L1',  'L2', 'L3', 'volumeLabels'],
                              'Rows' : [faRows, l1Rows, l2Rows, l3Rows, vlRows], 
                              'Columns' : [faColumns, l1Columns, l2Columns, l3Columns, vlColumns],
                              'Missing' : [faMissing, l1Missing, l2Missing, l3Missing, vlMissing]});
print("Table 1: Data Summary")
print(dataSummary)

Table 1: Data Summary
           Data  Rows  Columns  Missing
0            FA    87      170      322
1            L1    87      170      322
2            L2    87      170      322
3            L3    87      170      322
4  volumeLabels    87      170      322


In [99]:
def getMissing(dataset):
    missingColumns = []
    
    print('Column\tMissing Values')
    for i in range(0, dataset.shape[1]):
        missing = dataset.iloc[:,i].isnull().sum()
        if(missing > 0):
            missingColumns.append(dataset.columns[i])
            print('%s\t%d'%(dataset.columns[i], missing))
            
    return missingColumns

In [100]:
print('Dataset: FA\n')
faMissingColumns = getMissing(faDataRaw)
print('\n=======================\n')
print('Dataset: L1\n')
l1MissingColumns = getMissing(l1DataRaw)
print('\n=======================\n')
print('Dataset: L2\n')
l2MissingColumns = getMissing(l2DataRaw)
print('\n=======================\n')
print('Dataset: L3\n')
l3MissingColumns = getMissing(l3DataRaw)
print('\n=======================\n')
print('Dataset: Volume Labels\n')
vlMissingColumns = getMissing(vlDataRaw)

Dataset: FA

Column	Missing Values
SuperiorFronto-occipitalFasciculusLeft	6
UncinateFasciculusLeft	1
Red Nucleus left	21
SuperiorFronto-occipitalFasciculusRight	1
Red Nucleus right	17
GLOBUS PALLIDUS right	1
Mammillary body right	75
Mammillary body left	21
Hypothalamus E left	76
Hypothalamus E right	86
LVL_occipitalRight	12
Unused	5


Dataset: L1

Column	Missing Values
SuperiorFronto-occipitalFasciculusLeft	6
UncinateFasciculusLeft	1
Red Nucleus left	21
SuperiorFronto-occipitalFasciculusRight	1
Red Nucleus right	17
GLOBUS PALLIDUS right	1
Mammillary body right	75
Mammillary body left	21
Hypothalamus E left	76
Hypothalamus E right	86
LVL_occipitalRight	12
Unused	5


Dataset: L2

Column	Missing Values
SuperiorFronto-occipitalFasciculusLeft	6
UncinateFasciculusLeft	1
Red Nucleus left	21
SuperiorFronto-occipitalFasciculusRight	1
Red Nucleus right	17
GLOBUS PALLIDUS right	1
Mammillary body right	75
Mammillary body left	21
Hypothalamus E left	76
Hypothalamus E right	86
LVL_occipitalRight	12


## Limpeza

1. Backup dos dados
2. Excluir colunas irrelevantes ("Unnamed 1" e "Unnamed 2")
3. Renomear primeira coluna de "Unnamed 0" para "subject"
4. Criação da coluna "als", discriminando pacientes com e sem ELA
5. Criaçào da coluna "group", discriminando pacientes do grupo de controle e diferentes tipos 
de ELA (ELAs, C9orf72 e VAPB)
6. União dos 3 dataset em um único dataset

In [101]:
def dataClean(dataset, missingColumns, suffix):
    # Backup Data
    data = pd.DataFrame(dataset); 
    
    # Drop all columns with missing values
    data.drop(missingColumns, axis = 1, inplace = True);
    
    # Change first column name
    data.rename(columns = {'ID/Labls' : 'subject'}, inplace = True);
    
    # Add sufix to each feature name
    for i in range(1, data.shape[1]):
        colName = data.columns[i] + '_' + suffix
        data.rename(columns = {data.columns[i] : colName}, inplace = True)
        
    # Create two new features    
    ## New column: ALS
    ### Map Values
    #### Legend
    #### 0 = control
    #### 1 =  ALS confirmed
    data['als'] = 1
    data.loc[data['subject'].str.startswith('ctl'), 'als'] = 0
   
    ## New column: Group
    ### Map Values
    #### Legend
    #### 0 = control
    #### 1 = sporadic ALS
    #### 2 = c9o ALS
    #### 3 = vapb ALS
    data['group'] = 0
    data.loc[data['subject'].str.startswith('sals'), 'group'] = 1
    data.loc[data['subject'].str.startswith('c9o'), 'group'] = 2
    data.loc[data['subject'].str.startswith('vap'), 'group'] = 3
    
    return data

In [112]:
faData = dataClean(faDataRaw, faMissingColumns, 'fa')
l1Data = dataClean(l1DataRaw, l1MissingColumns, 'l1')
l2Data = dataClean(l2DataRaw, l2MissingColumns, 'l2')
l3Data = dataClean(l3DataRaw, l3MissingColumns, 'l3')
vlData = dataClean(vlDataRaw, vlMissingColumns, 'vl')

In [116]:
#l1Data.iloc[0,:]

newMetricsMD = l1Data.copy()
newMetricsRD = l1Data.copy()

for i in range(1, newMetricsMD.shape[1]):
    if(newMetricsMD.columns[i] != 'als' and newMetricsMD.columns[i] != 'group'):
        colName = newMetricsMD.columns[i][0:-3] + '_md'
        newMetricsMD.rename(columns = {newMetricsMD.columns[i] : colName}, inplace = True)
        
        colName = newMetricsRD.columns[i][0:-3] + '_rd'     
        newMetricsRD.rename(columns = {newMetricsRD.columns[i] : colName}, inplace = True)


for i in range(0, l1Data.shape[0]):
    for j in range(1, l1Data.shape[1]):
        newMetricsMD.iloc[i, j] = (l1Data.iloc[i, j] + 
                                   l2Data.iloc[i, j] + 
                                   l3Data.iloc[i, j])/3
        
        newMetricsRD.iloc[i, j] = (l2Data.iloc[i, j] + 
                                   l3Data.iloc[i, j])/2

In [114]:
# Merge all data into one dataframe
allData = pd.DataFrame(faData)
allData = allData.merge(l1Data, how = 'inner', on = ['subject', 'als', 'group'])
#allData = allData.merge(l2Data, how = 'inner', on = ['subject', 'als', 'group'])
#allData = allData.merge(l3Data, how = 'inner', on = ['subject', 'als', 'group'])
allData = allData.merge(newMetricsMD, how = 'inner', on = ['subject', 'als', 'group'])
allData = allData.merge(newMetricsRD, how = 'inner', on = ['subject', 'als', 'group'])
allData = allData.merge(vlData, how = 'inner', on = ['subject', 'als', 'group'])
allData

for i  in range(0, allData.shape[1]):
    print(allData.columns[i])

subject
SUPERIOR PARIETAL GYRUS left  (gm)_fa
CINGULATE GYRUS left  (gm)_fa
SUPERIOR FRONTAL GYRUS left (gm)_fa
MIDDLE FRONTAL GYRUS left (gm)_fa
INFERIOR FRONTAL GYRUS left (gm)_fa
PRECENTRAL GYRUS left (gm)_fa
POSTCENTRAL GYRUS left (gm)_fa
ANGULAR GYRUS left (gm)_fa
PRE-CUNEUS left (gm)_fa
CUNEUS left (gm)_fa
LINGUAL GYRUS left (gm)_fa
FUSIFORM GYRUS left  (gm)_fa
PARAHIPPOCAMPAL GYRUS left (gm)_fa
SUPERIOR OCCIPITAL GYRUS left (gm)_fa
INFERIOR OCCIPITAL GYRUS (gm)_fa
MIDDLE OCCIPITAL GYRUS (gm)_fa
ENTORHINAL AREA  (gm)_fa
SUPERIOR TEMPORAL GYRUS (gm)_fa
INFERIOR TEMPORAL GYRUS (gm)_fa
MIDDLE TEMPORAL GYRUS (gm)_fa
LATERAL FRONTO-ORBITAL GYRUS (gm)_fa
MIDDLE FRONTO-ORBITAL GYRUS (gm)_fa
SUPRAMARGINAL GYRUS (gm)_fa
GYRUS RECTUS (gm)_fa
INSULAR (gm)_fa
AMYGDALA_fa
HIPPOCAMPUS_fa
CEREBELLUM_fa
CorticospinalTractLeft_fa
InferiorCerebellarPeduncleLeft_fa
MedialLemniscusLeft_fa
SuperiorCerebellarPeduncleLeft_fa
CerebralPeduncleLeft_fa
AnteriorLimbInternalCapsuleLeft_fa
PosteriorLimbIntern

## Treinamento do Modelo

In [115]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import feature_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

In [117]:
# Define Helper functions

# Compute confusion matrix, sensitivity and specificity
def results(model, x, y):
    yPred = model.predict(x) # Model predictions
    CM = confusion_matrix(y, yPred) # Compute confusion Matrix
    Sens = CM[1,1]/(y == 1).sum() # Calculate Sensitivity
    Spec = CM[0,0]/(y == 0).sum() # Calculate Specificity
    
    print('matriz de confusão = \n', CM)
    print('Sensibilidade = ', Sens)
    print('Especificidade = ', Spec)
    print('acc = ', np.sum(y == yPred)/yPred.size)
    
# Show result for the different datasets
def showResults(model, name, x, y, xTrain, xVal, xTest, yTrain, yVal, yTest):
    title = 'Modelo: ' + name
    print(title)

    print('Conjunto de Treino\n')
    results(model, xTrain, yTrain)
    
    print('\nConjunto de Validação\n')
    results(model, xVal, yVal)
    
    print('\nConjunto de Teste\n')
    results(model, xTest, yTest)
    
    #Calculate Cross Validation
    k = 100
    print('Cross validation (%d-Fold):'%(k))
    cv = ShuffleSplit(n_splits = k, test_size = 0.5, random_state = 5)
    scores = cross_val_score(model, x, y, cv=cv)
    print('Score = ', scores.mean())
    print('Std = ', scores.std())
    
def prepareData(dataset, target, zeroVariance = 0, pca = 0):
    x = dataset.drop(['subject', 'als', 'group'], axis=1)
    y = dataset[target]

    print('Initial Dimensions: ', x.shape)
    if(zeroVariance > 0):
        sel = feature_selection.VarianceThreshold(threshold = zeroVariance)
        x = sel.fit_transform(x)
        print('Post zeroVar: ', x.shape)
    if(pca > 0):
        x = PCA(pca).fit_transform(x)
        print('Post pca: ', x.shape)

    xTrain, xTmp, yTrain, yTmp = train_test_split(x, y, test_size = 0.5, random_state = 5)
    xVal, xTest, yVal, yTest = train_test_split(xTmp, yTmp, test_size = 0.33, random_state = 5)

    xTrain = sm.add_constant(xTrain)
    xVal = sm.add_constant(xVal)
    xTest = sm.add_constant(xTest)

    return x, y, xTrain, yTrain, xVal, yVal, xTest, yTest    

### Classificador ELA/Saudavel

In [118]:
filteredData = allData

x, y, xTrain, yTrain, xVal, yVal, xTest, yTest = prepareData(filteredData, 'als', 0.1, 0.99)

clfAlsLR = make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', max_iter = 5000))
clfAlsSVM = make_pipeline(StandardScaler(), svm.SVC(gamma='auto', kernel='rbf'))
clfAlsRF = make_pipeline(StandardScaler(), RandomForestClassifier())

clfAlsLR.fit(xTrain, yTrain);
clfAlsSVM.fit(xTrain, yTrain);
clfAlsRF.fit(xTrain, yTrain);

Initial Dimensions:  (87, 785)
Post zeroVar:  (87, 157)
Post pca:  (87, 18)


In [119]:
showResults(clfAlsLR, 'Logistic Regression', x, y, xTrain, xVal, xTest, yTrain, yVal, yTest)
print('\n======================================================\n')
showResults(clfAlsSVM, 'Support Vector Machine', x, y, xTrain, xVal, xTest, yTrain, yVal, yTest)
print('\n======================================================\n')
showResults(clfAlsRF, 'Random Forests', x, y, xTrain, xVal, xTest, yTrain, yVal, yTest)

Modelo: Logistic Regression
Conjunto de Treino

matriz de confusão = 
 [[ 4  4]
 [ 1 34]]
Sensibilidade =  0.9714285714285714
Especificidade =  0.5
acc =  0.8837209302325582

Conjunto de Validação

matriz de confusão = 
 [[ 2  8]
 [ 1 18]]
Sensibilidade =  0.9473684210526315
Especificidade =  0.2
acc =  0.6896551724137931

Conjunto de Teste

matriz de confusão = 
 [[ 1  1]
 [ 2 11]]
Sensibilidade =  0.8461538461538461
Especificidade =  0.5
acc =  0.8
Cross validation (100-Fold):
Score =  0.7193181818181817
Std =  0.06407116072277477


Modelo: Support Vector Machine
Conjunto de Treino

matriz de confusão = 
 [[ 1  7]
 [ 0 35]]
Sensibilidade =  1.0
Especificidade =  0.125
acc =  0.8372093023255814

Conjunto de Validação

matriz de confusão = 
 [[ 0 10]
 [ 0 19]]
Sensibilidade =  1.0
Especificidade =  0.0
acc =  0.6551724137931034

Conjunto de Teste

matriz de confusão = 
 [[ 0  2]
 [ 0 13]]
Sensibilidade =  1.0
Especificidade =  0.0
acc =  0.8666666666666667
Cross validation (100-Fold):


### Classificador: Esporadico/C9o/VAPB

In [120]:
filteredData = allData[allData['als'] == 1]
x, y, xTrain, yTrain, xVal, yVal, xTest, yTest = prepareData(filteredData, 'group')

clfGroupSVM = make_pipeline(StandardScaler(), svm.SVC(gamma='auto', kernel='rbf'))
clfGroupRF = make_pipeline(StandardScaler(), RandomForestClassifier())

clfGroupSVM.fit(xTrain, yTrain);
clfGroupRF.fit(xTrain, yTrain);

Initial Dimensions:  (67, 785)


In [121]:
showResults(clfGroupSVM, 'Support Vector Machine', x, y, xTrain, xVal, xTest, yTrain, yVal, yTest)
print('\n======================================================\n')
showResults(clfGroupRF, 'Random Forests', x, y, xTrain, xVal, xTest, yTrain, yVal, yTest)

Modelo: Support Vector Machine
Conjunto de Treino

matriz de confusão = 
 [[13  0  0]
 [ 0  6  1]
 [ 0  0 13]]
Sensibilidade =  0.46153846153846156
Especificidade =  inf
acc =  0.9696969696969697

Conjunto de Validação

matriz de confusão = 
 [[4 0 6]
 [2 0 2]
 [3 0 5]]
Sensibilidade =  0.0
Especificidade =  inf
acc =  0.4090909090909091

Conjunto de Teste

matriz de confusão = 
 [[4 0 2]
 [1 0 0]
 [1 0 4]]
Sensibilidade =  0.0
Especificidade =  inf
acc =  0.6666666666666666
Cross validation (100-Fold):


<ipython-input-117-b10e28883bfc>:8: RuntimeWarning: divide by zero encountered in long_scalars
  Spec = CM[0,0]/(y == 0).sum() # Calculate Specificity
<ipython-input-117-b10e28883bfc>:8: RuntimeWarning: divide by zero encountered in long_scalars
  Spec = CM[0,0]/(y == 0).sum() # Calculate Specificity
<ipython-input-117-b10e28883bfc>:8: RuntimeWarning: divide by zero encountered in long_scalars
  Spec = CM[0,0]/(y == 0).sum() # Calculate Specificity


Score =  0.4802941176470588
Std =  0.09421456407076872


Modelo: Random Forests
Conjunto de Treino

matriz de confusão = 
 [[13  0  0]
 [ 0  7  0]
 [ 0  0 13]]
Sensibilidade =  0.5384615384615384
Especificidade =  inf
acc =  1.0

Conjunto de Validação

matriz de confusão = 
 [[7 2 1]
 [2 1 1]
 [2 1 5]]
Sensibilidade =  0.1
Especificidade =  inf
acc =  0.5909090909090909

Conjunto de Teste

matriz de confusão = 
 [[4 1 1]
 [0 1 0]
 [1 0 4]]
Sensibilidade =  0.16666666666666666
Especificidade =  inf
acc =  0.75
Cross validation (100-Fold):


<ipython-input-117-b10e28883bfc>:8: RuntimeWarning: divide by zero encountered in long_scalars
  Spec = CM[0,0]/(y == 0).sum() # Calculate Specificity
<ipython-input-117-b10e28883bfc>:8: RuntimeWarning: divide by zero encountered in long_scalars
  Spec = CM[0,0]/(y == 0).sum() # Calculate Specificity
<ipython-input-117-b10e28883bfc>:8: RuntimeWarning: divide by zero encountered in long_scalars
  Spec = CM[0,0]/(y == 0).sum() # Calculate Specificity


Score =  0.5367647058823529
Std =  0.08260203923519778


## Exportar Dados

In [111]:
# Interim
faData.to_csv('../data/interim/faData.csv', index=False)
l1Data.to_csv('../data/interim/l1Data.csv', index=False)
l2Data.to_csv('../data/interim/l2Data.csv', index=False)
l3Data.to_csv('../data/interim/l3Data.csv', index=False)
vlData.to_csv('../data/interim/vlData.csv', index=False)

# Processed
allData.to_csv('../data/processed/processedData.csv', index=False)